In [1]:
import pandas as pd
from connection import connect


In [2]:
#connection
co_oltp,etl_conn=connect('production')


['product_list_price_history', 'product_model', 'product_model_illustration', 'product_model_prod_desc_culture', 'product_subcategory', 'scrap_reason', 'transaction_history', 'transaction_history_archive', 'bill_of_materials', 'unit_measure', 'document', 'work_order', 'culture', 'illustration', 'location', 'product', 'product_category', 'product_cost_history', 'product_photo', 'product_product_photo', 'product_review', 'product_description', 'work_order_routing', 'product_document', 'product_inventory']


## Extract

In [9]:
#extract DimProductCategory


from deep_translator import GoogleTranslator

dim_product_subcategory=pd.read_sql_table('product_subcategory',co_oltp,schema='production')
# dim_product_subcategory


## Transform


In [ ]:
dim_product_subcategory=dim_product_subcategory.rename(columns={'product_subcategory_id':'product_subcategory_alternate_key','name':'english_product_subcategory_name'})

dim_product_subcategory=dim_product_subcategory.drop(['rowguid','modified_date'],axis=1)
dim_product_subcategory['spanish_product_subcategory_name']=dim_product_subcategory['english_product_subcategory_name'].apply(lambda x : GoogleTranslator(source='en',target='es').translate(x) if x is not None else None )
dim_product_subcategory['french_product_subcategory_name']=dim_product_subcategory['english_product_subcategory_name'].apply(lambda x : GoogleTranslator(source='en',target='fr').translate(x) if x is not None else None )

dim_product_category=pd.read_sql_table('dim_product_category',etl_conn)

dim_product_subcategory = dim_product_subcategory.merge(
    dim_product_category[['product_category_alternate_key', 'product_category_key']],
    left_on='product_category_id',
    right_on='product_category_alternate_key',
    how='left'
)

dim_product_subcategory.drop(['product_category_alternate_key','product_category_id'], inplace=True,axis=1)

dim_product_subcategory

,product_subcategory_alternate_key,product_category_id,english_product_subcategory_name,spanish_product_subcategory_name,french_product_subcategory_name,product_category_alternate_key,product_category_key
0,1,1,Mountain Bikes,Bicicletas de montaña,VTT,1,1
1,2,1,Road Bikes,Bicicletas de carretera,Vélos de route,1,1
2,3,1,Touring Bikes,Bicicletas de turismo,Vélos de tourisme,1,1
3,4,2,Handlebars,Bigote daliniano,Guidon,2,2
4,5,2,Bottom Brackets,Soportes inferiores,Supports inférieurs,2,2
5,6,2,Brakes,Frenos,Freins,2,2
6,7,2,Chains,Cadenas,Chaînes,2,2
7,8,2,Cranksets,Platos y bielas,Pédaliers,2,2
8,9,2,Derailleurs,Desviadores,Dérailleurs,2,2
9,10,2,Forks,Horquillas,Fourchettes,2,2


## Load

In [20]:
dim_product_subcategory.to_sql('dim_product_subcategory', etl_conn, if_exists='append',index=False)

37